In [1]:
from datetime import date
from data_manager import DataManager
from scipy.stats import median_abs_deviation
import pandas as pd


In [2]:
self = DataManager()

def get_average_team_stat(self, df, last_n_team_games):
    teams = self.query_teams()
    
    # Initialize columns outside the loop
    stats = ['fgm', 'fga', 'fg3m', 'fg3a', 'points', 'rebounds', 'assists', 'steals', 'blocks', "to"]
    for stat in stats:
        df.loc[:, f'team_{stat}_mad'] = None
        df.loc[:, f'team_{stat}_average'] = None

    for team in df['TEAM'].unique():
        team_id = [team_object.id for team_object in teams if team_object.nickname == team][0]
        print(team_id)
        team_games = self.get_and_save_team_data(team_id).sort_values(by='date', ascending=False).head(last_n_team_games)
        print(team_games.columns)
        for stat in stats:
            total_points_average = team_games[stat].mean()
            mad_points = median_abs_deviation(team_games[stat], scale="normal")
            df.loc[df['TEAM'] == team, f'team_{stat}_average'] = total_points_average
            df.loc[df['TEAM'] == team, f'team_{stat}_mad'] = mad_points
            
    return df

In [3]:

analyzed_props = self.get_analyzed_props()
analyzed_prop_records = pd.DataFrame.from_dict([prop.entry for prop in analyzed_props])
profitable_props = analyzed_prop_records[analyzed_prop_records['EV'] > 0]


Received raw input.
Loading points props.
Loading assists props.
Loading fg3m props.
Loading rebounds props.
Loading fgm props.
Loading steals props.
Loading blocks props.
                 player_name          team    stat  over_threshold  over_odds  \
0                Rudy Gobert  Timberwolves  points            10.5       -105   
1            Jaden McDaniels  Timberwolves  points             9.5       -130   
2            Anthony Edwards  Timberwolves  points            27.5       -130   
3                   Naz Reid  Timberwolves  points             8.5       -110   
4                Mike Conley  Timberwolves  points            11.5       -105   
..                       ...           ...     ...             ...        ...   
72              Aaron Gordon       Nuggets  blocks             0.5        115   
73        Michael Porter Jr.       Nuggets  blocks             0.5       -155   
74  Kentavious Caldwell-Pope       Nuggets  blocks             0.5        115   
75              Ni

In [4]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
view = profitable_props[['PLAYER', 'TEAM', 'STAT', 'TYPE']]
last_n_team_games = 25
df = get_average_team_stat(self, profitable_props, last_n_team_games=last_n_team_games)

14
Index(['team_name', 'points', 'rebounds', 'assists', 'efg', 'fg3a', 'fg3m',
       'fg3_pct', 'fga', 'fgm', 'fta', 'ft_pct', 'steals', 'blocks', 'to',
       'date'],
      dtype='object')
7
Index(['team_name', 'points', 'rebounds', 'assists', 'efg', 'fg3a', 'fg3m',
       'fg3_pct', 'fga', 'fgm', 'fta', 'ft_pct', 'steals', 'blocks', 'to',
       'date'],
      dtype='object')


C:\Users\rusta\AppData\Local\Temp\ipykernel_2112\4244349758.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'team_{stat}_mad'] = None
C:\Users\rusta\AppData\Local\Temp\ipykernel_2112\4244349758.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'team_{stat}_average'] = None
C:\Users\rusta\AppData\Local\Temp\ipykernel_2112\4244349758.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [5]:
print(df) 

df = df.sort_values(by="EV", ascending=False).reset_index(drop=True)
df.to_csv("temp.csv")
print(df.columns)


                       PLAYER          TEAM      STAT  THRESH  ODDS   TYPE  \
0                 Rudy Gobert  Timberwolves    points    10.5  -105   over   
2             Jaden McDaniels  Timberwolves    points     9.5  -130   over   
5             Anthony Edwards  Timberwolves    points    27.5  -105  under   
6                    Naz Reid  Timberwolves    points     8.5  -110   over   
8                 Mike Conley  Timberwolves    points    11.5  -105   over   
14               Jamal Murray       Nuggets    points    19.5  -120   over   
16         Michael Porter Jr.       Nuggets    points    13.5  -120   over   
21               Nikola Jokic       Nuggets    points    30.5  -125  under   
22                Rudy Gobert  Timberwolves   assists     1.5   100   over   
26            Anthony Edwards  Timberwolves   assists     5.5   120   over   
29         Karl-Anthony Towns  Timberwolves   assists     3.5  -180  under   
32                   Naz Reid  Timberwolves   assists     0.5  -

In [6]:
from collections import deque

teams = df['TEAM'].unique()
if len(teams) != 2:
    raise ValueError("The TEAM column must contain exactly two unique values for this operation.")

# Get the values to be swapped
value1 = df.loc[df['TEAM'] == teams[0], 'team_to_average'].values[0]
value2 = df.loc[df['TEAM'] == teams[1], 'team_to_average'].values[0]

# Create a mapping to swap the values
mapping = {teams[0]: value2, teams[1]: value1}

# Apply the mapping to the 'team_average_to' column
df['team_to_average'] = df['TEAM'].map(mapping)

# Rename the column
df.rename(columns={'team_to_average': 'oppteam_to_average'}, inplace=True)

    

In [7]:
team_stats_cols = [col for col in df.columns if "average" in col]
team_stats = df.copy()[team_stats_cols]
new_cols = [col.split("_")[1] for col in team_stats.columns]
team_stats.columns = new_cols
team_stats['TEAM'] = df['TEAM']


initial_availability = {}
for team in team_stats['TEAM'].unique():
    initial_availability[team] = dict(team_stats[team_stats['TEAM'] == team].reset_index(drop=True).iloc[0, :])



                

In [8]:

import copy

parlay_length = 6
parlay_count = 10
used = []
parlays = []
for i in range(parlay_count):
    availability = copy.deepcopy(initial_availability)
    parlay = []
    counter = 0
    for i, row in df.iterrows():
        if len(parlay) == parlay_length:
            break
        if i in used:
            continue
        else:
            if row['STAT'] in new_cols:
                stat = row['STAT']
                if row['TYPE'] == "over" and availability[row['TEAM']][stat] - (row['THRESH'] * 3) > 0:
                    availability[row['TEAM']][stat] -= row['THRESH']
                    parlay.append(i)
                    used.append(i)
                elif row['TYPE'] == "under" and availability[row['TEAM']][stat] + row['THRESH'] < initial_availability[row['TEAM']][stat]:
                    availability[row['TEAM']][stat] += row['THRESH']
                    parlay.append(i)
                    used.append(i)
    print(parlay)
    parlays.append(parlay)
parlays = [parlay for parlay in parlays if len(parlay) == parlay_length]
print(parlays)


                    

[0, 1, 2, 3, 4, 5]
[6, 8, 10, 11, 13, 14]
[16, 17, 19, 20, 21, 22]
[25, 26, 29, 32, 33, 34]
[35, 38, 43, 44, 45, 46]
[48, 51]
[]
[]
[]
[]
[[0, 1, 2, 3, 4, 5], [6, 8, 10, 11, 13, 14], [16, 17, 19, 20, 21, 22], [25, 26, 29, 32, 33, 34], [35, 38, 43, 44, 45, 46]]


In [9]:
parlay_dfs = []
# Segment the original DataFrame
for indexes in parlays:
    parlay_df = df.iloc[indexes]
    parlay_dfs.append(parlay_df)

# Display the resulting DataFrames
for i, segment_df in enumerate(parlay_dfs):
    print(f"Parlay {i}:\n{segment_df}\n")
    segment_df.to_csv(f"parlays/{date.today()}_{i}_greedy.csv")

Parlay 0:
               PLAYER          TEAM      STAT  THRESH  ODDS  TYPE      PROB  \
0            Naz Reid  Timberwolves    points     8.5  -110  over  0.942650   
1            Naz Reid  Timberwolves       fgm     3.5   130  over  0.767062   
2         Rudy Gobert  Timberwolves    points    10.5  -105  over  0.802129   
3            Naz Reid  Timberwolves      fg3m     1.5   150  over  0.625501   
4            Naz Reid  Timberwolves  rebounds     3.5  -120  over  0.791408   
5  Michael Porter Jr.       Nuggets    points    13.5  -120  over  0.780080   

         EV  HOUSE_PROB team_fgm_mad team_fgm_average team_fga_mad  \
0  3.998027    0.523810     2.965204            40.48     5.930409   
1  3.821213    0.434783     2.965204            40.48     5.930409   
2  2.830304    0.512195     2.965204            40.48     5.930409   
3  2.818766    0.400000     2.965204            40.48     5.930409   
4  2.254576    0.545455     2.965204            40.48     5.930409   
5  2.150729    0